In [1]:
!pip install pytorch
!pip install transformers
!pip install torchmetrics

  ERROR: Failed building wheel for pytorch
  Running setup.py clean for pytorch
Failed to build pytorch
    Running setup.py install for pytorch ... error
ERROR: Command errored out with exit status 1: /usr/bin/python3 -u -c 'import io, os, sys, setuptools, tokenize; sys.argv[0] = '"'"'/tmp/pip-install-c5lb2s38/pytorch_ca7bc4a9397740fd8e112f5341081ffd/setup.py'"'"'; __file__='"'"'/tmp/pip-install-c5lb2s38/pytorch_ca7bc4a9397740fd8e112f5341081ffd/setup.py'"'"';f = getattr(tokenize, '"'"'open'"'"', open)(__file__) if os.path.exists(__file__) else io.StringIO('"'"'from setuptools import setup; setup()'"'"');code = f.read().replace('"'"'\r\n'"'"', '"'"'\n'"'"');f.close();exec(compile(code, __file__, '"'"'exec'"'"'))' install --record /tmp/pip-record-7l6h1dmi/install-record.txt --single-version-externally-managed --compile --install-headers /usr/local/include/python3.7/pytorch Check the logs for full command output.
     |████████████████████████████████| 4.0 MB 8.7 MB/s 
     |████████████

In [2]:
import pandas as pd

from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score

import transformers
from transformers import AutoTokenizer, AdamW, RobertaForSequenceClassification

import torch
from torch.nn import functional as F
from torch.utils.data import DataLoader, Dataset
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score

from tqdm.notebook import tqdm


import numpy as np

In [3]:
from google.colab import drive
drive.mount('/content/drive/')

Mounted at /content/drive/


In [4]:
df = pd.read_csv('/content/drive/MyDrive/RoBERTa/dataset.csv')
df.head(5)

,Sentence,Emotion
0,언니 동생으로 부르는 게 맞는 일인가요..??,공포
1,그냥 내 느낌일 뿐이겠지?,공포
2,아직 너무 초기라서 그런 거죠?,공포
3,유치원 버스 사고 낫다던데,공포
4,근데 원래 이런 거 맞나요,공포


In [5]:
df_shuffled = df.sample(frac=1).reset_index(drop=True)
df_shuffled.head()

,Sentence,Emotion
0,이젠 일본 대중문화 쓰레기 수준 되었네 한국이 너무 발전했어,놀람
1,"무슨 색 무슨 꽃으로 해줘야 하지,",공포
2,우리 효자 귀는 요 이상형이 키 크고 성격 좋은 남자 거 등요?ㅋㅋㅋ,놀람
3,전 좀 춥네요.,중립
4,더디게 빠지는 게 참 힘드네요. ㅜ...,슬픔


In [6]:
df_shuffled.loc[(df_shuffled['Emotion'] == "공포"), 'Emotion'] = 0  #공포 => 0
df_shuffled.loc[(df_shuffled['Emotion'] == "놀람"), 'Emotion'] = 1  #놀람 => 1
df_shuffled.loc[(df_shuffled['Emotion'] == "분노"), 'Emotion'] = 2  #분노 => 2
df_shuffled.loc[(df_shuffled['Emotion'] == "슬픔"), 'Emotion'] = 3  #슬픔 => 3
df_shuffled.loc[(df_shuffled['Emotion'] == "중립"), 'Emotion'] = 4  #중립 => 4
df_shuffled.loc[(df_shuffled['Emotion'] == "행복"), 'Emotion'] = 5  #행복 => 5
df_shuffled.loc[(df_shuffled['Emotion'] == "혐오"), 'Emotion'] = 6  #혐오 => 6

In [7]:
train, val  = train_test_split(df_shuffled, test_size=0.25, random_state=42)

In [8]:
train = train[['Sentence','Emotion']]
val = val[['Sentence','Emotion']]

In [9]:
class NTDataset(Dataset):
  
    def __init__(self, csv_file):
        self.dataset = csv_file
        self.tokenizer = AutoTokenizer.from_pretrained("klue/roberta-large")

        print(self.dataset.describe())

    def __len__(self):
        return len(self.dataset)

    def __getitem__(self, idx):
        row = self.dataset.iloc[idx, 0:2].values
        text = row[0]
        y = row[1]
        inputs = self.tokenizer(
            text, 
            return_tensors='pt',
            truncation=True,
            max_length=64,
            padding='max_length',
            add_special_tokens=True
        )

        input_ids = inputs['input_ids'][0]
        attention_mask = inputs['attention_mask'][0]

        return input_ids, attention_mask, y

In [10]:
class NTDataset_test(Dataset):
  
    def __init__(self, csv_file):
        self.dataset = csv_file
        self.tokenizer = AutoTokenizer.from_pretrained("klue/roberta-large")

        print(self.dataset.describe())

    def __len__(self):
        return len(self.dataset)

    def __getitem__(self, idx):
        row = self.dataset.iloc[idx, 0:1].values
        text = row[0]
        inputs = self.tokenizer(
            text, 
            return_tensors='pt',
            truncation=True,
            max_length=64,
            padding='max_length',
            add_special_tokens=True
        )

        input_ids = inputs['input_ids'][0]
        attention_mask = inputs['attention_mask'][0]

        return input_ids, attention_mask

In [11]:
train_dataset = NTDataset(train)

val_dataset = NTDataset(val)

Downloading:   0%|          | 0.00/375 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/243k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/734k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/173 [00:00<?, ?B/s]

       Sentence  Emotion
count     28881    28881
unique    28872        7
top      감사합니다.        5
freq          2     4504
       Sentence  Emotion
count      9628     9628
unique     9627        7
top      임신인가요?        5
freq          2     1499


In [12]:
device = torch.device("cuda")

In [13]:
model = RobertaForSequenceClassification.from_pretrained("klue/roberta-large", num_labels=7).to(device)

Downloading:   0%|          | 0.00/547 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.25G [00:00<?, ?B/s]

Some weights of the model checkpoint at klue/roberta-large were not used when initializing RobertaForSequenceClassification: ['lm_head.decoder.weight', 'lm_head.decoder.bias', 'lm_head.dense.bias', 'lm_head.layer_norm.weight', 'lm_head.dense.weight', 'lm_head.bias', 'lm_head.layer_norm.bias']
- This IS expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of RobertaForSequenceClassification were not initialized from the model checkpoint at klue/roberta-large and are newly initialized: ['classifier.out_proj.weight', 'classifier.dense.bias', 'class

In [14]:
epochs = 5
batch_size = 32
#f1 = F1Score(num_classes=7 ,average='weighted',mdmc_average='samplewise')

In [15]:
optimizer = AdamW(model.parameters(), lr=1e-5)
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=batch_size, shuffle=True)

/usr/local/lib/python3.7/dist-packages/transformers/optimization.py:309: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  FutureWarning,


In [16]:
model = model.to(device)

In [17]:
# train


for i in range(epochs):
    losses = []
    accuracies = []
    total_loss = 0.0
    correct = 0
    total = 0
    f1 = [0,0,0,0,0,0,0]
    cnt = 0
    model.train()

    for input_ids_batch, attention_masks_batch, y_batch in tqdm(train_loader):
        optimizer.zero_grad()
        y_batch = y_batch.to(device)
        y_pred = model(input_ids_batch.to(device), attention_mask=attention_masks_batch.to(device))[0]
        loss = F.cross_entropy(y_pred, y_batch)
        loss.backward()
        optimizer.step()

        total_loss += loss.item()

        _, predicted = torch.max(y_pred, 1)
        correct += (predicted == y_batch).sum()
        f1 += f1_score(y_batch.cpu().detach().numpy(), predicted.cpu().detach().numpy(),labels=[0,1,2,3,4,5,6],average=None,zero_division=0)
        cnt += 1
        total += len(y_batch)

    losses.append(total_loss)
    accuracies.append(correct.float() / total)
    print("Train Loss:", total_loss / total, "Accuracy:", correct.float() / total, "f1_score:", f1/cnt)

  0%|          | 0/903 [00:00<?, ?it/s]

Train Loss: 0.03708833968068526 Accuracy: tensor(0.5434, device='cuda:0') f1_score: [0.53523139 0.50640202 0.46469018 0.57082754 0.33708439 0.7617063
 0.32046142]


  0%|          | 0/903 [00:00<?, ?it/s]

Train Loss: 0.029528443191129166 Accuracy: tensor(0.6401, device='cuda:0') f1_score: [0.64393543 0.60767718 0.52722429 0.6956364  0.48414962 0.84570975
 0.40739267]


  0%|          | 0/903 [00:00<?, ?it/s]

Train Loss: 0.023279237949341718 Accuracy: tensor(0.7194, device='cuda:0') f1_score: [0.73481025 0.68615696 0.58824385 0.7809169  0.60041695 0.89286953
 0.51180935]


  0%|          | 0/903 [00:00<?, ?it/s]

Train Loss: 0.017238147546720737 Accuracy: tensor(0.7954, device='cuda:0') f1_score: [0.80893396 0.78898693 0.65761602 0.85220535 0.73580836 0.93428355
 0.60689919]


  0%|          | 0/903 [00:00<?, ?it/s]

Train Loss: 0.01187637345069193 Accuracy: tensor(0.8611, device='cuda:0') f1_score: [0.87456026 0.86155578 0.73257764 0.91049426 0.83829988 0.95183175
 0.69677018]


In [18]:
# validation
model.eval()

pred = []
correct = 0
total = 0
f1 = [0,0,0,0,0,0,0]
cnt = 0

for input_ids_batch, attention_masks_batch, y_batch in tqdm(val_loader):
    y_batch = y_batch.to(device)
    y_pred = model(input_ids_batch.to(device), attention_mask=attention_masks_batch.to(device))[0]
    _, predicted = torch.max(y_pred, 1)
    pred.append(predicted)
    correct += (predicted == y_batch).sum()
    total += len(y_batch)
  
    f1 += f1_score(y_batch.cpu().detach().numpy(), predicted.cpu().detach().numpy(),labels=[0,1,2,3,4,5,6],average=None,zero_division=0)
    cnt += 1
    total += len(y_batch)

losses.append(total_loss)
accuracies.append(correct.float() / total)

print("val accuracy:", correct.float() / total, "f1_score:", f1/cnt)

  0%|          | 0/301 [00:00<?, ?it/s]

val accuracy: tensor(0.2869, device='cuda:0') f1_score: [0.55203138 0.5288825  0.47225296 0.6386238  0.38448847 0.79011844
 0.35387807]


In [19]:
def predict(predict_sentence):

    data = [predict_sentence]
    dataset_another = pd.DataFrame(data)

    another_test = NTDataset_test(dataset_another)
    test_dataloader = DataLoader(another_test, batch_size=batch_size, num_workers=5)
    
    model.eval()

    for input_ids_batch, attention_masks_batch in tqdm(test_dataloader):
        y_pred = model(input_ids_batch.to(device), attention_mask=attention_masks_batch.to(device))[0]
        _, predicted = torch.max(y_pred, 1)
        pred.extend(predicted.tolist())


        test_eval=[]
        for i in y_pred:
            logits=i
            logits = logits.detach().cpu().numpy()

            if np.argmax(logits) == 0:
                test_eval.append("공포가")
            elif np.argmax(logits) == 1:
                test_eval.append("놀람이")
            elif np.argmax(logits) == 2:
                test_eval.append("분노가")
            elif np.argmax(logits) == 3:
                test_eval.append("슬픔이")
            elif np.argmax(logits) == 4:
                test_eval.append("중립이")
            elif np.argmax(logits) == 5:
                test_eval.append("행복이")
            elif np.argmax(logits) == 6:
                test_eval.append("혐오가")

        print(">> 입력하신 내용에서 " + test_eval[0] + " 느껴집니다.")

In [20]:
#end = 1
#while end == 1 :
#    sentence = input("하고싶은 말을 입력해주세요 : ")
#    if sentence == "0" :
#        break
#    predict(sentence)
#    print("\n")